In [31]:
import requests
import json
import numpy as np
# GRID CAN BE prism, goes back to 1981
# if want to go further back, go back to 1950 with nrcc-nn
#change nrcc-model to prism or nrcc-nn
# if i remove county mean it just gives values and not counties
def get_rainfall_data(n,s,e,w,start_date, end_date):
    input_dict = {"bbox":f"{w},{s},{e},{n}","sdate":f"{start_date}","edate":f"{end_date}","grid":"nrcc-model","elems":[{"name":"pcpn","interval":[0,0,1]}]}
    # gets previous 30 days including current day and sums it = {"bbox":f"{w},{s},{e},{n}","date":"2008-07-30","grid":"nrcc-model","elems":[{"name":"pcpn","interval":[1,0,0],"duration":30,"reduce":"sum"}]}
    #input_dict = {"bbox":"-75,40,-74,41","sdate":"2023-07-01","edate":"2023-07-07","grid":"nrcc-model","elems":[{"name":"pcpn","interval":[0,0,1],"area_reduce":"county_mean"}]}
    req = requests.post("http://grid2.rcc-acis.org/GridData", json = input_dict)
    data = req.json()
    return data

def get_rainfall_data_avg(n,s,e,w, end_date, window):
    input_dict = {"bbox":f"{w},{s},{e},{n}",f"{end_date}":"2008-07-30","grid":"nrcc-model","elems":[{"name":"pcpn","interval":[1,0,0],"duration":window,"reduce":"sum"}]}
    req = requests.post("http://grid2.rcc-acis.org/GridData", json = input_dict)
    data = req.json()
    return data

def avg_rainfall_data(data):
    info = {}
    for dates in data["data"]:
        accum = 0
        total = 0
        for section in dates[1]:
            for item in section:
                if item != -999:
                    total += item
                    accum += 1
        if accum != 0:
            total /= accum
        info[dates[0]] = total
    return info

In [32]:
# graph_comparison(data[], "date", "start_date", "end_date", index_of_site, window_in_wks, if_plotting, percentile) & daily_analysis()
# daily_analysis(graph[], start(dt), end(dt), "date", "end_date", start2(dt))
# change in gw levels for one station, sorted from decrease to increase
import matplotlib.pyplot as plt
import numpy as np

MONTH_DATA = [31,28,31,30,31,30,31,31,30,31,30,31,31]
# Avg every day
import datetime as dt
MON_ACCUM = [0, 31, 59, 90, 120, 151, 181, 212, 243, 273, 304, 334]

def daily_analysis(graph, start, end, date, end_date, start2):
    # returns list of that week average over all years
    date_data = []
    for date in list(graph.keys()):
        #print(str(graph(date)))
        date_data.append([dt.datetime(int(date[0:4]),int(date[5:7]),int(date[8:10])), graph[date]])
    total_data = []
    while start.year < int(end_date):
        lst = []
        accum = 0
        for i in date_data:
            if i[0] >= start and i[0] <= end:
                lst.append(i)
                accum += i[1]
        if len(lst):
            total_data.append([start2.year, accum / len(lst)])
        #if start.year % 4 == 0 and start.month == 2:
            # LEAP YEAR
        start = dt.datetime(start.year + 1, start.month, start.day)
        start2 = dt.datetime(start2.year + 1, start2.month, start2.day)
        end = dt.datetime(end.year + 1, end.month, end.day)
    return total_data

def daily_no_avg(graph, start, date, end_date, start2, scope, typ):
    # returns list of that week average over all years
    date_data = []
    for date in list(graph.keys()):
        #print(str(graph(date)))
        date_data.append([dt.datetime(int(date[0:4]),int(date[5:7]),int(date[8:10])), graph[date]])
    total_data = []
    # just take first day and next first day
    while start.year < int(end_date):
        #print(f'start is {start}')
        lst2 = []
        accum = 0
        min = 1000000
        for i in range(len(date_data)):
            #print(date_data[i])
            #print(start)
            #print(date_data[i][0])
            #print(start)
            if date_data[i][0] >= start - dt.timedelta(days=scope) and date_data[i][0] <= start + dt.timedelta(days=scope):
                #print(date_data[i][0])
                lst2.append(date_data[i])
                if typ == 'avg':
                    #print(date_data[i][1])
                    accum += date_data[i][1]
                elif typ == "min" and date_data[i][1] < min:
                    min = date_data[i][1]
        if len(lst2) >= 2:
            if typ == "avg":
                total_data.append([start2.year, accum/len(lst2)])
            elif typ == "min":
                total_data.append([start2.year, min])
        #if start.year % 4 == 0 and start.month == 2:
            # LEAP YEAR
        start = dt.datetime(start.year + 1, start.month, start.day)
        start2 = dt.datetime(start2.year + 1, start2.month, start2.day)
    return total_data

def sortkey2(e):
    return e[1]

def graph_comparison(graph, date, start_date, end_date, index_of_site, window, plotting, percentile, scope, typ):
    if len(date) == 5:
        start = dt.datetime(int(start_date), int(date[0:2]), int(date[3:5]))
        start2 = start + dt.timedelta(weeks=window * 2)
        end2 = start + dt.timedelta(weeks=3 * window,days=-1)
    #print(graph[0])
    #print(graph[1])
    first_set = daily_no_avg(graph[0], start, date, end_date, start, scope, typ)
    second_set = daily_no_avg(graph[1], start2, date, end_date, start, scope, typ)
    #print(first_set)
    #print(second_set)
    plottable = []
    for i in first_set:
        for j in second_set:
             if j[0] == i[0]:
                  plottable.append([i[0], j[1] - i[1]])
    dct = {}
    plottable.sort(key=sortkey2)
    for i in plottable:
        if plotting:
            dct[(str(i[0]))[2:4]] = i[1]
        else:
            dct[(str(i[0]))] = i[1]
    if plotting:
        plt.bar(dct.keys(), dct.values(), color = "green", width = 0.8)
        plt.xlabel("Year")
        plt.ylabel("Value")
        plt.title("Rainfall (%s) versus period after" % (date))
        plt.show()
        #for i in dct.keys():
            #print(f'{i}: {dct[i]:.2f}')
    else:
        #print(f'dct is {dct}')
        if len(dct) > 0:
            if percentile != 0:
                a = list(dct.values())
                a = np.array(a)
                p = np.percentile(a, percentile)  # return 50th percentile, i.e. median.
                return p
            else:
                # [mean, median, interquartile range,standard deviation]
                b = list(dct.values())
                b = np.array(b)
                return [list(dct.values()), np.mean(b), np.median(b), np.percentile(b, 75) - np.percentile(b, 25), np.std(b),start.strftime("%m/%d"), end2.strftime("%m/%d"), list(dct.keys())]
        else:
            return False

In [33]:
# using_all_weeks(data[], "date", "start_date", "end_date", index_of_site, percent_of_total_outputted)
def sortkey(e):
    return e[0]

def using_all_weeks(data, date, start_date, end_date, index_of_site, percent, scope, typ):
    graph = data
    date = dt.datetime(int(start_date), int(date[0:2]), int(date[3:5]))
    window = 0.5
    total_data = []
    date_data = []
    for week in range(int(26 / window) + 1):
        date_data.append({})
    for stat in graph:
        new_date = dt.datetime(int(stat[0:4]),int(stat[5:7]),int(stat[8:10]))
        date_data[((new_date.timetuple().tm_yday - 1) // 7)][stat] = graph[stat]
    for week in range(int(26 / window)):
        date2 = date + dt.timedelta(weeks= week * 2 * window)
        # investigate looking across year
        #Convert to string
        mo = str(date2.month)
        da = str(date2.day)
        if len(mo) == 1:
            mo = "0" + mo
        if len(da) == 1:
            da = '0' + da
        #print(graph_comparison(data, f"{mo}-{da}", start_date, end_date, index_of_site, window, True, 0))
        #graph_comparison(graph, f"{mo}-{da}", start_date, end_date, index_of_site, window, True, 0)
        info = (graph_comparison([{**date_data[week - 1], **date_data[week], **date_data[week + 1]}, {**date_data[week], **date_data[week + 1]}], f"{mo}-{da}", start_date, end_date, index_of_site, window, False, 0, scope, typ))
        #print(info)
        # Need to gather full date of entry
        #ERROR
        if info:
            for i in range(len(info[0])):
                total_data.append([info[0][i],f"{info[7][i]}/{(info[5])} - {info[7][i]}/{info[6]}"])
    total_data.sort(key=sortkey)
    print(str(percent) + "%")
    for i in range(int(len(total_data) / (100 / percent))):
        print(f'{total_data[i][0]:.2f}: {total_data[i][1]}')

    print(str(100 - percent) + "%")
    for i in total_data[-1 * int(len(total_data) / (100 / percent)): -1]:
        print(f'{i[0]:.2f}: {i[1]}')
    return total_data[0:(int(int(len(total_data) / (100 / percent))))]
    #print("This data may be skewed because when only looking at one day compared to the next week day, if either day is missing, the\ndata gets tossed out. Therefore, plenty of years where there are great changes may be left out.\nThis can be solved by looking at each day of the week, starting at 01-01 to 01-06.")

In [34]:
# Driver
#avg_rainfall_data(get_rainfall_data(41,40,-73,-74,"2019-12-15","2020-1-23"))
#data = get_rainfall_data(41,40,-73,-74,"1950-01-01","2022-12-31")
#data = avg_rainfall_data(data)
#using_all_weeks(data, "01-01", "1950","2022", 0, 2, 2, "avg")

'''

ill have 3 lists of dates, how do i sort?
sort but still keep separate?

iterate through one list type:
    if month and year match, print all of those saying streamflow or evap or rainfall
    
go to end of one list


from rainfall:
for first time period, want 30 day that ends on the day the first chunk ends. Not precip for 30 days, but departure from normal (will get a call for that)
30 day period is bucket, bucket is normally that full, but this year how much from normal fullness

summation of 30 days in year i - normal

30 days 60 days or 90 days
use same bounding boxes as clusters for stream flow analysis

top 5 big dates
other period, summation of 30 days (gap can be 1 week or 2 weeks or 3 weeks apart) (2 or 3 weeks mostly)
jan30-feb30-mar30-ect for all time
stream flow avg over 7 days

2%
7 day avg change with next 7 days: sum of all rainfall of last 30 days ending on end of 7 day avg

'''

 1 weeks apart
-1.09, 2007/04/29 - 2007/05/13 matches -32000.00, 2007/04/02 - 2007/04/16
 2 weeks apart
-1.75, 2005/04/01 - 2005/04/22 matches -1733.00, 2005/04/02 - 2005/04/23
 3 weeks apart
 4 weeks apart


'\n\nill have 3 lists of dates, how do i sort?\nsort but still keep separate?\n\niterate through one list type:\n    if month and year match, print all of those saying streamflow or evap or rainfall\n    \ngo to end of one list\n\n\nfrom rainfall:\nfor first time period, want 30 day that ends on the day the first chunk ends. Not precip for 30 days, but departure from normal (will get a call for that)\n30 day period is bucket, bucket is normally that full, but this year how much from normal fullness\n\nsummation of 30 days in year i - normal\n\n30 days 60 days or 90 days\nuse same bounding boxes as clusters for stream flow analysis\n\ntop 5 big dates\nother period, summation of 30 days (gap can be 1 week or 2 weeks or 3 weeks apart) (2 or 3 weeks mostly)\njan30-feb30-mar30-ect for all time\nstream flow avg over 7 days\n\n2%\n7 day avg change with next 7 days: sum of all rainfall of last 30 days ending on end of 7 day avg\n\n'